In [1]:
import getpass
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [2]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

In [3]:
llm = ChatGroq(temperature=0, model="llama3-70b-8192")

In [4]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate

In [18]:
tools = [TavilySearchResults(max_results=1)]

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un ayudante muy útil. Asegúrese de utilizar la herramienta tavily_search_results_json para obtener información.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}, Contesta en español."),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [19]:
# Construir el Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': [], 'agent_scratchpad': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Eres un ayudante muy útil. Asegúrese de utilizar

In [16]:
# Crear un agente ejecutor pasando el agente y las herramientas
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': [], 'agent_scratchpad': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_vari

In [20]:
resultado = agent_executor.invoke({"input": "¿Qué es LangChain?"})
resultado



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '¿Qué es LangChain?'}`


[{'url': 'https://en.wikipedia.org/wiki/LangChain', 'content': 'In October 2023 LangChain introduced LangServe, a deployment tool designed to facilitate the transition from LCEL (LangChain Expression Language) prototypes to production-ready applications.[5]\nIntegrations[edit]\nAs of March 2023, LangChain included integrations with systems including Amazon, Google, and Microsoft Azure cloud storage; API wrappers for news, movie information, and weather; Bash for summarization, syntax and semantics checking, and execution of shell scripts; multiple web scraping subsystems and templates; few-shot learning prompt generation support; finding and summarizing "todo" tasks in code; Google Drive documents, spreadsheets, and presentations summarization, extraction, and creation; Google Search and Microsoft Bing web search; OpenAI, Anthropic, and Hugging Face language models; 

{'input': '¿Qué es LangChain?',
 'output': 'LangChain es un proyecto de código abierto lanzado en octubre de 2022 que se describe como un "marco de integración de modelos de lenguaje" que permite a los desarrolladores crear aplicaciones más inteligentes y automatizar tareas utilizando modelos de lenguaje como OpenAI, Anthropic y Hugging Face. LangChain proporciona una forma de integrar modelos de lenguaje con otras herramientas y servicios, como bases de datos, APIs y sistemas de archivos, lo que permite a los desarrolladores crear aplicaciones más poderosas y flexibles.'}